<a href="https://colab.research.google.com/github/Panperception/QKD/blob/main/QRC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step-by-Step Process
1. Fetching Stock Price Data
We’ll use the yfinance library to download historical closing prices for a stock, such as Apple (AAPL), over a specific period (e.g., 2020 to 2023). This gives us a realistic time-series dataset.
2. Preprocessing the Data
Stock prices vary widely in scale compared to a sine wave, so we need to preprocess them:
Normalization: Scale the prices to a range like [-1, 1] based on the dataset’s minimum and maximum values, then further adjust to [-π, π] to suit the quantum circuit’s rotation gates.
Sequences and Targets: Create sequences of length T (e.g., 5 days) from the normalized data, with the next value (e.g., the 6th day) as the target for prediction.
3. Quantum Reservoir Circuit
The QRC model uses a quantum circuit to process each sequence:
Input Encoding: At each time step, encode the current stock price value into a qubit (e.g., qubit 0) using an RZ gate, scaled by a factor like π.
Random Rotations: Apply fixed random RY rotations to all qubits to create a reservoir-like dynamics.
Mixing: Use CNOT gates to entangle the qubits, spreading the input information across the system.
This process repeats for each time step in the sequence.
4. Feature Extraction
After processing the sequence, simulate the circuit and compute the expectation values of the Pauli-Z operator for each qubit. These values serve as features for the prediction model.
5. Training and Evaluation
Use the features to train a simple linear regression model to predict the next stock price. Evaluate the model’s performance using the mean squared error (MSE) between the predictions and actual targets.


## Example Code
Here’s a complete Python script to test the QRC model on stock price data:

In [ ]:
!pip install 'qiskit==1'
!pip install qiskit-aer
!pip install qiskit-terra[visualization]


In [4]:
import numpy as np
import yfinance as yf
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from qiskit.opflow import I, Z #This line imports I and Z from the necessary module

# Parameters
n_qubits = 4  # Number of qubits in the reservoir
T = 5         # Length of the input sequence
alpha = np.pi  # Scaling factor for input encoding

# Fetch stock data (Apple closing prices from 2020-2023)
stock_data = yf.download('AAPL', start='2020-01-01', end='2023-01-01')['Close'].values

# Normalize to [-1, 1] and scale to [-π, π]
stock_data_normalized = (stock_data - np.min(stock_data)) / (np.max(stock_data) - np.min(stock_data)) * 2 - 1
stock_data_scaled = stock_data_normalized * np.pi

# Create sequences and targets
sequences = [stock_data_scaled[i:i+T] for i in range(len(stock_data_scaled) - T)]
targets = [stock_data_scaled[i+T] for i in range(len(stock_data_scaled) - T)]

# Generate random angles for RY gates (fixed for the reservoir)
np.random.seed(42)  # For reproducibility
theta = np.random.uniform(-np.pi, np.pi, size=n_qubits)

# Define the quantum circuit for a sequence
def build_circuit(seq, theta):
    qc = QuantumCircuit(n_qubits)
    for t in range(T):
        # Encode input: RZ rotation on qubit 0
        qc.rz(alpha * seq[t][0], 0) # Access the numerical value using [0]
        # Apply random RY rotations to all qubits
        for j in range(n_qubits):
            qc.ry(theta[j], j)
        # Apply CNOT chain for mixing
        for j in range(n_qubits - 1):
            qc.cx(j, j+1)
    return qc

# Simulate and get expectation values for Z on each qubit
features = []
for seq in sequences:
    qc = build_circuit(seq, theta)
    state = Statevector.from_instruction(qc)
    # Compute <Z> for each qubit
    expectations = [state.expectation_value(I.tensorpower(i) ^ Z ^ I.tensorpower(n_qubits - i - 1)) for i in range(n_qubits)] #Changed this line
    features.append(expectations)

features = np.array(features)

# Train linear regression model
model = LinearRegression()
model.fit(features, targets)

# Predict and evaluate on training data
predictions = model.predict(features)
mse = mean_squared_error(targets, predictions)
print(f"Training MSE: {mse}")

ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.

### Explanation of the Code
* Libraries: We use yfinance for stock data, qiskit for quantum simulation, and sklearn for linear regression and evaluation.
* Data Fetching: Downloads Apple’s closing prices from 2020 to 2023.
* Normalization: Scales the data to [-π, π] for quantum encoding.
* Quantum Circuit: Processes each sequence of T=5 values, encoding them step-by-step and mixing with random rotations and CNOTs.
* Features: Extracts expectation values from the final quantum state.
* Model: Trains a linear regression model and computes the MSE on the training data.

### Notes and Considerations
* Scalability: This example uses a small number of qubits (n_qubits=4) and sequence length (T=5) to keep the simulation manageable on a classical computer. Larger systems may require quantum hardware or optimized simulators.
* Performance: The random reservoir might not capture complex stock price patterns perfectly. You could experiment with more qubits, longer sequences, or a different reservoir structure.
* Evaluation: This code evaluates on training data for simplicity. For a more robust test, split the data into training and testing sets.
* Flexibility: You can easily modify the stock ticker (e.g., 'MSFT' for Microsoft) or time period in the yf.download call.

### Conclusion
Testing a QRC model on stock price data is not only feasible but also a great way to explore its potential with real-world time series. The example above demonstrates how to fetch, preprocess, and model stock data using a quantum reservoir, providing a practical starting point. Feel free to tweak the parameters or dataset to suit your needs!